In [16]:
SEED=42
# SEEDS = [42, 0, 2000,]
SEEDS = [42]

import os
import glob
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
import scipy as sc
from sklearn.model_selection import KFold, GroupKFold
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns', 300)
pd.set_option('max_rows', 300)

# Function to read our base train and test set
def read_train_test():
    train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
    test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')
    # Create a key to merge with book and trade data
    train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
    test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)
    print(f'Our training set has {train.shape[0]} rows')
    return train, test

# Function to get group stats for the stock_id and time_id
def get_time_stock(df):
    # Get realized volatility columns
    vol_cols = ['log_return1_realized_volatility', 'log_return2_realized_volatility', 'log_return1_realized_volatility_450', 'log_return2_realized_volatility_450', 
                'log_return1_realized_volatility_300', 'log_return2_realized_volatility_300', 'log_return1_realized_volatility_150', 'log_return2_realized_volatility_150', 
                'trade_log_return_realized_volatility', 'trade_log_return_realized_volatility_450', 'trade_log_return_realized_volatility_300', 'trade_log_return_realized_volatility_150']

    # Group by the stock id
    df_stock_id = df.groupby(['stock_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', 'median']).reset_index()
    # Rename columns joining suffix
    df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
    df_stock_id = df_stock_id.add_suffix('_' + 'stock')

    # Group by the time id
    df_time_id = df.groupby(['time_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', 'median']).reset_index()
    # Rename columns joining suffix
    df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
    df_time_id = df_time_id.add_suffix('_' + 'time')
    
    # Merge with original dataframe
    df = df.merge(df_stock_id, how = 'left', left_on = ['stock_id'], right_on = ['stock_id__stock'])
    df = df.merge(df_time_id, how = 'left', left_on = ['time_id'], right_on = ['time_id__time'])
    df.drop(['stock_id__stock', 'time_id__time'], axis = 1, inplace = True)
    return df

# Function to calculate the root mean squared percentage error
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

In [2]:
train, _ = read_train_test()
df_book = pd.read_csv('../input/processed-book-ffill/df_book.csv')
print(df_book.shape)
df_trade = pd.read_csv('../input/processed-book-ffill/df_trade.csv')
print(df_trade.shape)
train_ = df_book.merge(df_trade, on = ['row_id'], how = 'left')
train = train.merge(train_, on = ['row_id'], how = 'left')

# Get group stats of time_id and stock_id
train = get_time_stock(train)
train['logtarget'] = train['target'].apply(np.log)
train

Our training set has 428932 rows
(428932, 169)
(428913, 41)


,stock_id,time_id,target,row_id,wap1_sum,wap1_mean,wap1_std,wap1_median,wap2_sum,wap2_mean,wap2_std,wap2_median,log_return1_sum,log_return1_realized_volatility,log_return1_mean,log_return1_std,log_return1_median,log_return2_sum,log_return2_realized_volatility,log_return2_mean,log_return2_std,log_return2_median,wap_balance_sum,wap_balance_mean,wap_balance_std,wap_balance_median,price_spread_sum,price_spread_mean,price_spread_std,price_spread_median,bid_spread_sum,bid_spread_mean,bid_spread_std,bid_spread_median,ask_spread_sum,ask_spread_mean,ask_spread_std,ask_spread_median,total_volume_sum,total_volume_mean,total_volume_std,total_volume_median,volume_imbalance_sum,volume_imbalance_mean,volume_imbalance_std,volume_imbalance_median,wap1_sum_450,wap1_mean_450,wap1_std_450,wap1_median_450,wap2_sum_450,wap2_mean_450,wap2_std_450,wap2_median_450,log_return1_sum_450,log_return1_realized_volatility_450,log_return1_mean_450,log_return1_std_450,log_return1_median_450,log_return2_sum_450,log_return2_realized_volatility_450,log_return2_mean_450,log_return2_std_450,log_return2_median_450,wap_balance_sum_450,wap_balance_mean_450,wap_balance_std_450,wap_balance_median_450,price_spread_sum_450,price_spread_mean_450,price_spread_std_450,price_spread_median_450,bid_spread_sum_450,bid_spread_mean_450,bid_spread_std_450,bid_spread_median_450,ask_spread_sum_450,ask_spread_mean_450,ask_spread_std_450,ask_spread_median_450,total_volume_sum_450,total_volume_mean_450,total_volume_std_450,total_volume_median_450,volume_imbalance_sum_450,volume_imbalance_mean_450,volume_imbalance_std_450,volume_imbalance_median_450,wap1_sum_300,wap1_mean_300,wap1_std_300,wap1_median_300,wap2_sum_300,wap2_mean_300,wap2_std_300,wap2_median_300,log_return1_sum_300,log_return1_realized_volatility_300,log_return1_mean_300,log_return1_std_300,log_return1_median_300,log_return2_sum_300,log_return2_realized_volatility_300,log_return2_mean_300,log_return2_std_300,log_return2_median_300,wap_balance_sum_300,wap_balance_mean_300,wap_balance_std_300,wap_balance_median_300,price_spread_sum_300,price_spread_mean_300,price_spread_std_300,price_spread_median_300,bid_spread_sum_300,bid_spread_mean_300,bid_spread_std_300,bid_spread_median_300,ask_spread_sum_300,ask_spread_mean_300,ask_spread_std_300,ask_spread_median_300,total_volume_sum_300,total_volume_mean_300,total_volume_std_300,total_volume_median_300,volume_imbalance_sum_300,volume_imbalance_mean_300,volume_imbalance_std_300,volume_imbalance_median_300,wap1_sum_150,wap1_mean_150,wap1_std_150,wap1_median_150,wap2_sum_150,wap2_mean_150,wap2_std_150,wap2_median_150,log_return1_sum_150,log_return1_realized_volatility_150,log_return1_mean_150,log_return1_std_150,log_return1_median_150,log_return2_sum_150,log_return2_realized_volatility_150,log_return2_mean_150,log_return2_std_150,log_return2_median_150,wap_balance_sum_150,wap_balance_mean_150,...,trade_seconds_in_bucket_count_unique_450,trade_size_mean_450,trade_size_median_450,trade_size_std_450,trade_size_sum_450,trade_order_count_mean_450,trade_order_count_median_450,trade_order_count_std_450,trade_order_count_sum_450,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_mean_300,trade_size_median_300,trade_size_std_300,trade_size_sum_300,trade_order_count_mean_300,trade_order_count_median_300,trade_order_count_std_300,trade_order_count_sum_300,trade_log_return_realized_volatility_150,trade_seconds_in_bucket_count_unique_150,trade_size_mean_150,trade_size_median_150,trade_size_std_150,trade_size_sum_150,trade_order_count_mean_150,trade_order_count_median_150,trade_order_count_std_150,trade_order_count_sum_150,log_return1_realized_volatility_mean_stock,log_return1_realized_volatility_std_stock,log_return1_realized_volatility_max_stock,log_return1_realized_volatility_min_stock,log_return1_realized_volatility_median_stock,log_return2_realized_volatility_mean_stock,log_return2_realized_volatility_std_stock,log_return2_realized_volatility_max_stock,l

In [17]:
feature_cols = [c for c in train.columns if c not in ['row_id', 'target', 'time_id', 'stock_id', 'target', 'logtarget']]
wap1_cols = [c for c in feature_cols if c.split('_')[0]=='wap1']
feature_cols = [c for c in feature_cols if c not in wap1_cols]
print(f"# features: {len(feature_cols)}")

params = {
    'random_state': SEED,
    'objective': 'rmse',  
    'boosting_type': 'gbdt',
    'num_leaves': 100,
    'learning_rate': 0.1,
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'n_estimators': 10000,
    'min_child_samples': int(0.01*train.shape[0])
}

scores_by_seed = []
for j, s in enumerate(SEEDS):
    print(f"shuffle train with seed {s}")
    train_shuffled = train.sample(frac=1, random_state=s).reset_index(drop=True)
    oof_predictions = np.zeros(train.shape[0])
    kfold = GroupKFold(n_splits=5)
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(train_shuffled, groups=train_shuffled.time_id)):
        print(f'Training fold {fold}')
        x_train, x_val = train_shuffled.loc[trn_ind, feature_cols], train_shuffled.loc[val_ind, feature_cols]
        y_train, y_val = train_shuffled.loc[trn_ind, 'target'], train_shuffled.loc[val_ind, 'target']
        
        # Root mean squared percentage error weights
        train_weights = 1 / np.square(y_train)
        val_weights = 1 / np.square(y_val)

        # Fit with sklearn API
        model = lgb.LGBMRegressor(**params)
        model.fit(x_train, 
                  y_train, 
                  sample_weight=train_weights,
                  eval_set=[(x_val, y_val)],
                  eval_sample_weight=[val_weights],
                  eval_metric='rmse',
                  early_stopping_rounds=100,
                  verbose=False)

        # Add predictions to the out of folds array
        oof_predictions[val_ind] = model.predict(x_val)
    oof_score = rmspe(train_shuffled['target'], oof_predictions)
    scores_by_seed.append(oof_score)

    print(f"seed {s} oof rmspe: {oof_score}") # 0.2262, 0.2263, 0.2260
print(f"score averaged by {len(SEEDS)} seeds: {np.mean(scores_by_seed)}")

# features: 328
shuffle train with seed 42
Training fold 0
Training fold 1
Training fold 2
Training fold 3
Training fold 4
seed 42 oof rmspe: 0.22625313580807158
score averaged by seed: 0.22625313580807158


In [8]:
df_importance = pd.DataFrame({"feature": model.feature_name_, "importance": model.feature_importances_})
df_importance = df_importance.sort_values(by=["importance"], ascending=False).reset_index(drop=True)
df_importance

,feature,importance
0,log_return1_realized_volatility,138
1,trade_log_return_realized_volatility_450_media...,133
2,log_return1_realized_volatility_450_median_stock,109
3,log_return2_realized_volatility_450_min_time,106
4,trade_log_return_realized_volatility_450_min_time,92
...,...,...
323,wap_balance_std_450,0
324,wap_balance_sum_450,0
325,log_return2_median_450,0
326,log_return1_median_450,0
